In [1]:
!apt-get -qq -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1 > /dev/null
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!pip -q install https://github.com/kmaehashi/chainer-colab/releases/download/2018-02-06/cupy_cuda80-4.0.0b3-cp36-cp36m-linux_x86_64.whl
!pip -q install 'chainer==4.0.0b3'
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null
!pip -q install chainerrl
!pip -q install gym
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay
!pip install scikit-image

Extracting templates from packages: 100%


In [0]:
# !wget https://www.dropbox.com/s/ak394rqfx00r4cr/facades.zip
# !unzip facades.zip

In [0]:
from __future__ import print_function
import os
import argparse
from os.path import join
import numpy as np
from os import listdir
from os.path import join
from scipy.misc import imread, imresize, imsave

import chainer
import chainer.links as L
from chainer import cuda
import chainer.functions as F
from chainer import training
from chainer.training import extensions
from chainer import serializers
from chainer.utils import force_array
from chainer import optimizers, cuda, serializers
from chainer import Variable
from chainer.dataset import dataset_mixin

chainerの元からついてるmnistのデータの形にも注意。普通の画像の形にはなってない。

In [0]:
class MLP(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)  # n_units -> n_out

    def __call__(self, x, t):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return F.softmax_cross_entropy(self.l3(h2), t)
      
    def predict(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return F.softmax(self.l3(h2))

In [0]:
def main1():
    batch_size = 128
    num_classes = 10
    epochs = 1
    chanel = 1
    val_interval = (10 if False else 100000), 'iteration'
    log_interval = (10 if False else 1000), 'iteration'
    print(type(val_interval))
    # x_train, x_test = chainer.datasets.get_mnist(dtype=cuda.cupy)
    x_train, x_test = chainer.datasets.get_mnist()
    train_iter = chainer.iterators.SerialIterator(x_train, batch_size)
    test_iter = chainer.iterators.SerialIterator(x_test, batch_size,
                                                     repeat=False, shuffle=False)
    chainer.cuda.get_device_from_id(0).use()
    
    #モデル作成
    mlp_model = MLP(chanel, num_classes)
    mlp_model.to_gpu()

    optimizer = chainer.optimizers.Adam(alpha=0.0002, beta1=0.5)
    optimizer.setup(mlp_model)

    updater = training.StandardUpdater(train_iter, optimizer, device=0)
    trainer = training.Trainer(updater, (epochs, 'epoch'), "./")
    trainer.extend(extensions.PrintReport([
        'epoch', 'iteration', 'main/loss', 'validation/main/loss',
        'main/accuracy', 'validation/main/accuracy', 'lr'
    ]), trigger=log_interval)
    trainer.extend(extensions.ProgressBar(update_interval=10))
    trainer.run()
    
    #予測
    xp = cuda.cupy
    for x_train_ in train_iter:
      print(x_train_[0][0].shape)
      x = x_train_[0][0].reshape(784, 1).transpose(1, 0)
      print(mlp_model.predict(xp.asarray(x)))
      break


In [11]:
if __name__ == '__main__':
    main1()

<class 'tuple'>
     total [#.................................................]  2.13%
this epoch [#.................................................]  2.13%
        10 iter, 0 epoch / 1 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [##................................................]  4.27%
this epoch [##................................................]  4.27%
        20 iter, 0 epoch / 1 epochs
     174.8 iters/sec. Estimated time to finish: 0:00:02.567169.
     total [###...............................................]  6.40%
this epoch [###...............................................]  6.40%
        30 iter, 0 epoch / 1 epochs
    182.07 iters/sec. Estimated time to finish: 0:00:02.409828.
     total [####..............................................]  8.53%
this epoch [####..............................................]  8.53%
        40 iter, 0 epoch / 1 epochs
    183.55 iters/sec. Estimated time to finish: 0:00:02.335895.
     total [#####......

     total [#############################.....................] 59.73%
this epoch [#############################.....................] 59.73%
       280 iter, 0 epoch / 1 epochs
    182.96 iters/sec. Estimated time to finish: 0:00:01.031643.
     total [##############################....................] 61.87%
this epoch [##############################....................] 61.87%
       290 iter, 0 epoch / 1 epochs
     183.1 iters/sec. Estimated time to finish: 0:00:00.976230.
     total [################################..................] 64.00%
this epoch [################################..................] 64.00%
       300 iter, 0 epoch / 1 epochs
    183.18 iters/sec. Estimated time to finish: 0:00:00.921225.
     total [#################################.................] 66.13%
this epoch [#################################.................] 66.13%
       310 iter, 0 epoch / 1 epochs
    183.43 iters/sec. Estimated time to finish: 0:00:00.865440.
     total [####################

# sklearnでデータを作成

trainerを使うのであればVariableはつけなくても中でやってくれる。Chainerのtrainerでは(学習データ、正解ラベル)のタプルをリストにしてインテレータに渡す。

In [7]:
from sklearn import datasets, model_selection, svm, metrics
from sklearn.cross_validation import train_test_split
mnist = datasets.fetch_mldata('MNIST original', data_home='./')

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [0]:
dataset = np.asarray(mnist.data, np.float32)
dataset = dataset /255
target = np.asarray(mnist.target, np.int32)
data_train, data_test, label_train, label_test = train_test_split(dataset, target, test_size=0.2)
data_train = data_train.reshape((len(data_train), 1, 28, 28))
data_test = data_test.reshape((len(data_test), 1, 28, 28))

x_train = []
x_test = []
for (train, label) in zip(data_train, label_train):
  x_train.append((train,label))
for (test, label) in zip(data_test, label_test):
  x_test.append((test,label))

# print(x_train[0][0].shape)
# print(x_train[0][1])

In [0]:
class Cnn(chainer.Chain):
    def __init__(self, n_units, n_out):
      super(Cnn, self).__init__()
      with self.init_scope():
        #Convolution2D(self, in_channels, out_channels, ksize=None, stride=1, pad=0, nobias=False, initialW=None, initial_bias=None)
        self.conv1 = L.Convolution2D(None, 32, 3)
        self.conv2 = L.Convolution2D(None, 32*2, 3)
        self.conv3 = L.Convolution2D(None, 32*3, 3)
        #ksizeを入れないとエラー
#         self.conv1 = L.Convolution2D(None, 64)
#         self.conv2 = L.Convolution2D(None, 64 * 2)
#         self.conv3 = L.Convolution2D(None, 64 * 3)
#         self.loss1_conv = L.Convolution2D(None, 128, 1)
        self.loss1_fc1 = L.Linear(None, n_out)

    def __call__(self, input_cnn, t):
        cnn1 = F.relu(self.conv1(input_cnn))
        cnn2 = F.relu(self.conv2(cnn1))
        cnn3 = F.relu(self.conv3(cnn2))
        fc1 = self.loss1_fc1(cnn3)
        loss = F.softmax_cross_entropy(fc1, t)
        #accuracy
        accuracy = F.accuracy(fc1, t)
        chainer.report({
            'loss': loss,
            'accuracy': accuracy
        }, self)
        return loss
      
    def predict(self, input_cnn):
        cnn1 = F.relu(self.conv1(input_cnn))
        cnn2 = F.relu(self.conv2(cnn1))
        cnn3 = F.relu(self.conv3(cnn2))
        fc1 = self.loss1_fc1(cnn3)
        return F.softmax(fc1)

In [0]:
def main2():
    batch_size = 128
    num_classes = 10
    epochs = 1
    chanel = 1
    val_interval = (100 if True else 100000), 'iteration'
    log_interval = (100 if True else 1000), 'iteration'
    print(type(val_interval))
#     x_train, x_test = chainer.datasets.get_mnist(rgb_format=False)
    train_iter = chainer.iterators.SerialIterator(x_train, batch_size)
    test_iter = chainer.iterators.SerialIterator(x_test, batch_size,
                                                     repeat=False, shuffle=False)
    chainer.cuda.get_device_from_id(0).use()
    for x_train_ in train_iter:
      print(x_train_[0][0].shape)
      break
    
    #モデル作成
    mlp_model = Cnn(chanel, num_classes)
    mlp_model.to_gpu()

    optimizer = chainer.optimizers.Adam(alpha=0.0002, beta1=0.5)
    optimizer.setup(mlp_model)

    updater = training.StandardUpdater(train_iter, optimizer, device=0)
    trainer = training.Trainer(updater, (epochs, 'epoch'), "./")
    trainer.extend(extensions.LogReport(trigger=log_interval))
    trainer.extend(extensions.observe_lr(), trigger=log_interval)
    trainer.extend(extensions.Evaluator(test_iter, mlp_model, device=0), trigger=val_interval)
    trainer.extend(extensions.PrintReport([
        'epoch', 'iteration', 'main/loss', 'validation/main/loss',
        'main/accuracy', 'validation/main/accuracy', 'lr'
    ]), trigger=log_interval)
    trainer.extend(extensions.ProgressBar(update_interval=100))
    trainer.run()
    
    #予測
    xp = cuda.cupy
    for x_train_ in train_iter:
      print(x_train_[0][0].shape)
      x = x_train_[0][0].reshape(1, 28, 28, 1).transpose(3, 0, 1, 2)
      print(mlp_model.predict(xp.asarray(x)))
      break


In [55]:
if __name__ == '__main__':
    main2()

<class 'tuple'>
(1, 28, 28)
epoch       iteration   main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  lr        
0           100         0.475861    0.226339              0.866875       0.932315                  6.17115e-05  
     total [###########.......................................] 23.09%
this epoch [###########.......................................] 23.09%
       100 iter, 0 epoch / 1 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
0           200         0.143337    0.10831               0.959063       0.969484                  8.51707e-05  
     total [######################............................] 45.94%
this epoch [######################............................] 45.94%
       200 iter, 0 epoch / 1 epochs
    17.666 iters/sec. Estimated time to finish: 0:00:13.386965.
0           300         0.094525    0.0869222             0.972891       0.974526                  0.000101842  
     total [##################################

カーネルサイズ（受容野）ksizeの変更

In [0]:
class Cnn2(chainer.Chain):
    def __init__(self, n_units, n_out):
      super(Cnn2, self).__init__()
      with self.init_scope():
        #Convolution2D(self, in_channels, out_channels, ksize=None, stride=1, pad=0, nobias=False, initialW=None, initial_bias=None)
        self.conv1 = L.Convolution2D(None, 32, 1)
        self.conv2 = L.Convolution2D(None, 32*2, 3)
        self.conv3 = L.Convolution2D(None, 32*3, 3)
        self.loss1_fc1 = L.Linear(None, n_out)

    def __call__(self, input_cnn, t):
        cnn1 = F.relu(self.conv1(input_cnn))
        cnn2 = F.relu(self.conv2(cnn1))
        cnn3 = F.relu(self.conv3(cnn2))
        fc1 = self.loss1_fc1(cnn3)
        loss = F.softmax_cross_entropy(fc1, t)
        #accuracy
        accuracy = F.accuracy(fc1, t)
        chainer.report({
            'loss': loss,
            'accuracy': accuracy
        }, self)
        return loss
      
    def predict(self, input_cnn):
        cnn1 = F.relu(self.conv1(input_cnn))
        cnn2 = F.relu(self.conv2(cnn1))
        cnn3 = F.relu(self.conv3(cnn2))
        fc1 = self.loss1_fc1(cnn3)
        return F.softmax(fc1)

In [66]:
def main3():
    batch_size = 128
    num_classes = 10
    epochs = 1
    chanel = 1
    val_interval = (100 if True else 100000), 'iteration'
    log_interval = (100 if True else 1000), 'iteration'
    train_iter = chainer.iterators.SerialIterator(x_train, batch_size)
    test_iter = chainer.iterators.SerialIterator(x_test, batch_size,
                                                     repeat=False, shuffle=False)
    chainer.cuda.get_device_from_id(0).use()
    #モデル作成
    mlp_model = Cnn2(chanel, num_classes)
    mlp_model.to_gpu()

    optimizer = chainer.optimizers.Adam(alpha=0.0002, beta1=0.5)
    optimizer.setup(mlp_model)

    updater = training.StandardUpdater(train_iter, optimizer, device=0)
    trainer = training.Trainer(updater, (epochs, 'epoch'), "./")
    trainer.extend(extensions.LogReport(trigger=log_interval))
    trainer.extend(extensions.observe_lr(), trigger=log_interval)
    trainer.extend(extensions.Evaluator(test_iter, mlp_model, device=0), trigger=val_interval)
    trainer.extend(extensions.PrintReport([
        'epoch', 'iteration', 'main/loss', 'validation/main/loss',
        'main/accuracy', 'validation/main/accuracy', 'lr'
    ]), trigger=log_interval)
    trainer.extend(extensions.ProgressBar(update_interval=100))
    trainer.run()
    
    #予測
    xp = cuda.cupy
    for x_train_ in train_iter:
      print(x_train_[0][0].shape)
      x = x_train_[0][0].reshape(1, 28, 28, 1).transpose(3, 0, 1, 2)
      print(mlp_model.predict(xp.asarray(x)))
      break

if __name__ == '__main__':
    main3()

epoch       iteration   main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  lr        
0           100         0.423277    0.20837               0.880703       0.94188                   6.17115e-05  
     total [###########.......................................] 22.86%
this epoch [###########.......................................] 22.86%
       100 iter, 0 epoch / 1 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
0           200         0.135527    0.109774              0.959766       0.967945                  8.51707e-05  
     total [######################............................] 45.71%
this epoch [######################............................] 45.71%
       200 iter, 0 epoch / 1 epochs
    15.566 iters/sec. Estimated time to finish: 0:00:15.257199.
0           300         0.103145    0.0858165             0.970391       0.975166                  0.000101842  
     total [##################################................] 68.57%
thi

# VGG

VGG64をmnistに適応しようと思ったらpaddingを入れまくらないとプーリングできない。結果的に発散して全然うまくいかない。224 × 224*3のデータのスケールにすごい最適化されてるんじゃないのかな？なので全体的にスケールを小さくモデルを作ればいいのでは。

In [0]:
#https://arxiv.org/pdf/1409.1556.pdf
#これの真似。
class VGG64(chainer.Chain):
    def __init__(self, n_units, n_out):
      super(VGG64, self).__init__()
      with self.init_scope():
        #Convolution2D(self, in_channels, out_channels, ksize=None, stride=1, pad=0, nobias=False, initialW=None, initial_bias=None)
        self.conv1 = L.Convolution2D(None, 64, 3)
        self.conv2 = L.Convolution2D(None, 128, 3)
        self.conv3 = L.Convolution2D(None, 256, 3)
        self.conv4 = L.Convolution2D(None, 256, 3)
        self.conv5 = L.Convolution2D(None, 512, 3)
        self.conv6 = L.Convolution2D(None, 512, 3)
        self.conv7 = L.Convolution2D(None, 512, 3)
        self.conv8 = L.Convolution2D(None, 512, 3)
        self.fc1 = L.Linear(None, 256)
        self.fc2 = L.Linear(None, 128)
        self.fc3 = L.Linear(None, n_out)

    def __call__(self, input_cnn, t):
        h = self.conv1(input_cnn)
        h = F.max_pooling_2d(h, ksize=2, stride=2, pad=2)
        h = self.conv2(h)
        h = F.max_pooling_2d(h, ksize=2, stride=2, pad=2)
#         print(h.shape)
        h = self.conv3(h)
        h = self.conv4(h)
        h = F.max_pooling_2d(h, ksize=2, stride=2, pad=2)
#         print(h.shape)
        h = self.conv5(h)
        h = self.conv6(h)
        h = F.max_pooling_2d(h, ksize=2, stride=2, pad=5)
        h = self.conv7(h)
        h = self.conv8(h)
        h = F.max_pooling_2d(h, ksize=2, stride=2, pad=2)
        h = self.fc1(h)
        h = self.fc2(h)
        h = self.fc3(h)
        loss = F.softmax_cross_entropy(h, t)
        #accuracy
        accuracy = F.accuracy(h, t)
        chainer.report({
            'loss': loss,
            'accuracy': accuracy
        }, self)
        return loss
      
    def predict(self, input_cnn):
        h = self.conv1(input_cnn)
        h = F.max_pooling_2d(h, ksize=2, stride=2, pad=2)
        h = self.conv2(h)
        h = F.max_pooling_2d(h, ksize=2, stride=2, pad=2)
#         print(h.shape)
        h = self.conv3(h)
        h = self.conv4(h)
        h = F.max_pooling_2d(h, ksize=2, stride=2, pad=2)
#         print(h.shape)
        h = self.conv5(h)
        h = self.conv6(h)
        h = F.max_pooling_2d(h, ksize=2, stride=2, pad=5)
        h = self.conv7(h)
        h = self.conv8(h)
        h = F.max_pooling_2d(h, ksize=2, stride=2, pad=2)
        h = self.fc1(h)
        h = self.fc2(h)
        h = self.fc3(h)
        return F.softmax(h)

In [88]:
def main3():
    batch_size = 128
    num_classes = 10
    epochs = 1
    chanel = 1
    val_interval = (100 if True else 100000), 'iteration'
    log_interval = (100 if True else 1000), 'iteration'
    train_iter = chainer.iterators.SerialIterator(x_train, batch_size)
    test_iter = chainer.iterators.SerialIterator(x_test, batch_size,
                                                     repeat=False, shuffle=False)
    chainer.cuda.get_device_from_id(0).use()
    #モデル作成
    model = VGG64(chanel, num_classes)
    model.to_gpu()

    optimizer = chainer.optimizers.Adam(alpha=0.0002, beta1=0.5)
    optimizer.setup(model)

    updater = training.StandardUpdater(train_iter, optimizer, device=0)
    trainer = training.Trainer(updater, (epochs, 'epoch'), "./")
    trainer.extend(extensions.LogReport(trigger=log_interval))
    trainer.extend(extensions.observe_lr(), trigger=log_interval)
    trainer.extend(extensions.Evaluator(test_iter, model, device=0), trigger=val_interval)
    trainer.extend(extensions.PrintReport([
        'epoch', 'iteration', 'main/loss', 'validation/main/loss',
        'main/accuracy', 'validation/main/accuracy', 'lr'
    ]), trigger=log_interval)
    trainer.extend(extensions.ProgressBar(update_interval=100))
    trainer.run()
    
    #予測
    xp = cuda.cupy
    for x_train_ in train_iter:
      print(x_train_[0][0].shape)
      x = x_train_[0][0].reshape(1, 28, 28, 1).transpose(3, 0, 1, 2)
      print(model.predict(xp.asarray(x)))
      break

if __name__ == '__main__':
    main3()

epoch       iteration   main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  lr        
0           100         nan         nan                   0.095          0.0990294                 6.17115e-05  
     total [###########.......................................] 22.86%
this epoch [###########.......................................] 22.86%
       100 iter, 0 epoch / 1 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
0           200         nan         nan                   0.100938       0.0990294                 8.51707e-05  
     total [######################............................] 45.71%
this epoch [######################............................] 45.71%
       200 iter, 0 epoch / 1 epochs
     9.317 iters/sec. Estimated time to finish: 0:00:25.490922.
0           300         nan         nan                   0.0996094      0.0990294                 0.000101842  
     total [##################################................] 68.57%
thi

In [0]:
# class Cnn(chainer.Chain):
#     def __init__(self, input_nc, output_nc, ngf):
#       super(Cnn, self).__init__()
#       with self.init_scope():
#         #input_nc=3, output_nc=3
#         #ngf=64
#         #Convolution2D(self, in_channels, out_channels, ksize=None, stride=1, pad=0, nobias=False, initialW=None, initial_bias=None)
#         self.conv1 = L.Convolution2D(input_nc, ngf, 4, 2, 1)
#         self.conv2 = L.Convolution2D(ngf, ngf * 2, 4, 2, 1)
#         self.conv3 = L.Convolution2D(ngf * 2, ngf * 4, 4, 2, 1)
#         self.conv4 = L.Convolution2D(ngf * 4, ngf * 8, 4, 2, 1)
#         self.conv5 = L.Convolution2D(ngf * 8, ngf * 8, 4, 2, 1)
#         self.conv6 = L.Convolution2D(ngf * 8, ngf * 8, 4, 2, 1)
#         self.conv7 = L.Convolution2D(ngf * 8, ngf * 8, 4, 2, 1)
#         self.conv8 = L.Convolution2D(ngf * 8, ngf * 8, 4, 2, 1)
#         self.batch_norm = L.BatchNormalization(ngf)
#         self.batch_norm2 = L.BatchNormalization(ngf * 2)
#         self.batch_norm4 = L.BatchNormalization(ngf * 4)
#         self.batch_norm8 = L.BatchNormalization(ngf * 8)
#         self.dconv1 = L.Deconvolution2D(ngf * 8, ngf * 8, 4, 2, 1)
#         self.dconv2 = L.Deconvolution2D(ngf * 8 * 2, ngf * 8, 4, 2, 1)
#         self.dconv3 = L.Deconvolution2D(ngf * 8 * 2, ngf * 8, 4, 2, 1)
#         self.dconv4 = L.Deconvolution2D(ngf * 8 * 2, ngf * 8, 4, 2, 1)
#         self.dconv5 = L.Deconvolution2D(ngf * 8 * 2, ngf * 4, 4, 2, 1)
#         self.dconv6 = L.Deconvolution2D(ngf * 4 * 2, ngf * 2, 4, 2, 1)
#         self.dconv7 = L.Deconvolution2D(ngf * 2 * 2, ngf, 4, 2, 1)
#         self.dconv8 = L.Deconvolution2D(ngf * 2, output_nc, 4, 2, 1)

#     def __call__(self, input_encdec):
#         # Encoder
#         # Convolution layers:
#         # input_encdec is (nc) x 256 x 256
#         e1 = self.conv1(input_encdec)
#         # state size is (ngf) x 128 x 128
#         e2 = self.batch_norm2(self.conv2(F.leaky_relu(e1)))
#         # state size is (ngf x 2) x 64 x 64
#         e3 = self.batch_norm4(self.conv3(F.leaky_relu(e2)))
#         # state size is (ngf x 4) x 32 x 32
#         e4 = self.batch_norm8(self.conv4(F.leaky_relu(e3)))
#         # state size is (ngf x 8) x 16 x 16
#         e5 = self.batch_norm8(self.conv5(F.leaky_relu(e4)))
#         # state size is (ngf x 8) x 8 x 8
#         e6 = self.batch_norm8(self.conv6(F.leaky_relu(e5)))
#         # state size is (ngf x 8) x 4 x 4
#         e7 = self.batch_norm8(self.conv7(F.leaky_relu(e6)))
#         # state size is (ngf x 8) x 2 x 2
#         # No batch norm on output of Encoder
#         e8 = self.conv8(F.leaky_relu(e7))

#         # Decoder
#         # Deconvolution layers:
#         # state size is (ngf x 8) x 1 x 1
#         d1_ = F.dropout(self.batch_norm8(self.dconv1(F.relu(e8))))
#         # state size is (ngf x 8) x 2 x 2
#         d1 = F.concat((d1_, e7), axis=1)
#         d2_ = F.dropout(self.batch_norm8(self.dconv2(F.relu(d1))))
#         # state size is (ngf x 8) x 4 x 4
#         d2 = F.concat((d2_, e6), axis=1)
#         d3_ = F.dropout(self.batch_norm8(self.dconv3(F.relu(d2))))
#         # state size is (ngf x 8) x 8 x 8
#         d3 = F.concat((d3_, e5), axis=1)
#         d4_ = self.batch_norm8(self.dconv4(F.relu(d3)))
#         # state size is (ngf x 8) x 16 x 16
#         d4 = F.concat((d4_, e4), axis=1)
#         d5_ = self.batch_norm4(self.dconv5(F.relu(d4)))
#         # state size is (ngf x 4) x 32 x 32
#         d5 = F.concat((d5_, e3), axis=1)
#         d6_ = self.batch_norm2(self.dconv6(F.relu(d5)))
#         # state size is (ngf x 2) x 64 x 64
#         d6 = F.concat((d6_, e2), axis=1)
#         d7_ = self.batch_norm(self.dconv7(F.relu(d6)))
#         # state size is (ngf) x 128 x 128
#         d7 = F.concat((d7_, e1), axis=1)
#         d8 = self.dconv8(F.relu(d7))
#         # state size is (nc) x 256 x 256
#         output = F.tanh(d8)
#         return output